In [0]:
%sql
USE CATALOG catproject_catalog;
USE SCHEMA ecom_bronze;

In [0]:
%sql
CREATE TABLE ecom_bronze.order_raw (
  order_id           STRING,
  customer_id        STRING,
  order_date         TIMESTAMP,
  status             STRING,
  order_items_json   STRING,
  items_total_amount DeCIMAL(18,2),
  total_amount       DECIMAL(18,2),
  discount_amount    DECIMAL(18,2),
  tax_amount         DECIMAL(18,2),
  shipping_fee       DECIMAL(18,2),
  payment_method     STRING,
  shipping_address   STRING,
  billing_address    STRING,
  source_system      STRING,
  ingestion_time     TIMESTAMP,
  ingestion_year     INT,
  ingestion_month    INT,
  operation_type     STRING,
  record_hash        STRING,
  source_updated_at  TIMESTAMP
)
USING DELTA
PARTITIONED BY (ingestion_year, ingestion_month);


Tạo Comment cho các cột

In [0]:
%sql
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN order_id COMMENT 'Mã đơn hàng duy nhất';
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN customer_id COMMENT 'Mã khách hàng';
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN order_date COMMENT 'Ngày đặt hàng';
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN status COMMENT 'Trạng thái đơn hàng';
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN order_items_json COMMENT 'Danh sách các sản phẩm trong đơn hàng';
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN total_amount COMMENT 'Tổng tiền đơn hàng';
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN discount_amount COMMENT 'Số tiền giảm giá';
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN tax_amount COMMENT 'Thuế';
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN shipping_fee COMMENT 'Phí vận chuyển';
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN payment_method COMMENT 'Phương thức thanh toán';
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN shipping_address COMMENT 'Địa chỉ giao hàng';
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN billing_address COMMENT 'Địa chỉ thanh toán';
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN source_system COMMENT 'Nguồn dữ liệu gốc';
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN ingestion_time COMMENT 'Thời điểm ingest';
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN ingestion_year COMMENT 'Năm ingest';
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN ingestion_month COMMENT 'Tháng ingest';
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN operation_type COMMENT 'Loại tác vụ: insert/update/delete';
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN record_hash COMMENT 'Hash toàn bộ record để kiểm tra thay đổi';
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN source_updated_at COMMENT 'Thời gian cập nhật cuối từ hệ thống nguồn';


Tạo Constraint cho các cột

In [0]:
%sql
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN order_id SET NOT NULL;
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN customer_id SET NOT NULL;
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN order_date SET NOT NULL;
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN status SET NOT NULL;
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN order_items_json SET NOT NULL;
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN items_total_amount SET NOT NULL;
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN total_amount SET NOT NULL;
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN discount_amount SET NOT NULL;
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN tax_amount SET NOT NULL;
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN shipping_fee SET NOT NULL;
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN payment_method SET NOT NULL;
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN shipping_address SET NOT NULL;
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN billing_address SET NOT NULL;
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN source_system SET NOT NULL;
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN ingestion_time SET NOT NULL;
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN ingestion_year SET NOT NULL;
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN ingestion_month SET NOT NULL;
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN operation_type SET NOT NULL;
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN record_hash SET NOT NULL;
ALTER TABLE ecom_bronze.order_raw ALTER COLUMN source_updated_at SET NOT NULL;

In [0]:
%sql
-- Ngày đặt hàng không ở tương lai
ALTER TABLE ecom_bronze.order_raw 
ADD CONSTRAINT valid_order_date CHECK (order_date <= current_timestamp());

-- Trạng thái đơn hàng hợp lệ
ALTER TABLE ecom_bronze.order_raw 
ADD CONSTRAINT valid_status CHECK (status IN ('pending', 'processing', 'shipped', 'delivered', 'cancelled'));

-- Tổng tiền không âm
ALTER TABLE ecom_bronze.order_raw 
ADD CONSTRAINT valid_total_amount CHECK (total_amount >= 0);

ALTER TABLE ecom_bronze.order_raw 
ADD CONSTRAINT valid_items_total_amount CHECK (items_total_amount >= 0);

-- Khuyến mãi không âm
ALTER TABLE ecom_bronze.order_raw 
ADD CONSTRAINT valid_discount_amount CHECK (discount_amount >= 0);

-- Thuế không âm
ALTER TABLE ecom_bronze.order_raw 
ADD CONSTRAINT valid_tax_amount CHECK (tax_amount >= 0);

-- Phí vận chuyển không âm
ALTER TABLE ecom_bronze.order_raw 
ADD CONSTRAINT valid_shipping_fee CHECK (shipping_fee >= 0);

-- Kiểm tra cách thanh toán hợp lệ
ALTER TABLE ecom_bronze.order_raw
ADD CONSTRAINT valid_payment_method
CHECK (
  payment_method IN ('cash', 'credit_card', 'bank_transfer', 'e_wallet')
);

-- kiểm tra ingestion time không ở tương lai
ALTER TABLE ecom_bronze.order_raw 
ADD CONSTRAINT valid_ingestion_time CHECK (ingestion_time <= current_timestamp());





Kích hoạt CDF

In [0]:
%sql
ALTER TABLE ecom_bronze.order_raw SET TBLPROPERTIES (delta.enableChangeDataFeed = true);

In [0]:
%sql
DESCRIBE EXTENDED order_raw